<a href="https://colab.research.google.com/github/RegRom/agency-swarm-lab/blob/master/Agency_Swarm_Advanced_Features_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
!pip install agency-swarm gradio

In [ ]:
from getpass import getpass
from agency_swarm import set_openai_key
import os
os.environ["OPENAI_API_KEY"] = getpass("Your OpenAI API Key: ")

# Async Mode
When it comes to asynchronous execution, there are 2 modes you can use at the moment: `threading`, `tools_threading`.

## Threading

If you would like to use asynchronous communication between agents, you can specify a `async_mode` parameter to `threading`. This is useful when you don't want to wait for a response from an agent. For example, if it takes it long to write it.

In [ ]:
from agency_swarm.tools import BaseTool
from pydantic import Field
import time

class MyCustomTool(BaseTool):
    example_field: str = Field(..., description="Description of the example field.")

    def run(self):
        time.sleep(5)  # Wait for 5 seconds
        return f"Processed {self.example_field}"

In [ ]:
from agency_swarm import Agent

ceo = Agent(name="CEO",
            description="Responsible for managing tasks.",
            instructions="You are the CEO. Delegate tasks to other agents.",
            tools=[MyCustomTool])  # CEO with two tools

essay_writer = Agent(name="Essay Writer",
                     description="Responsible for writing long-form essays.",
                     instructions="Write comprehensive and well-researched essays.",
                     tools=[])


In [ ]:
from agency_swarm import Agency

agency = Agency([
    ceo,
    [ceo, essay_writer],
], async_mode='threading')  # Enable asynchronous communication mode

In [ ]:
agency.run_demo()

## Tools Threading
If you would like to use asynchronous execution for tools, you can specify a `async_mode` parameter to `tools_threading`. With this mode on, all tools will be executed concurrently in separate threads, which can significantly speed up the work flow of I/O bound tasks.

In [ ]:
class AnotherCustomTool(BaseTool):
    another_field: int = Field(..., description="An integer field for this tool.")

    def run(self):
        time.sleep(25)  # Wait for 25 seconds
        return f"Processed integer {self.another_field}"

In [ ]:
ceo = Agent(name="CEO",
            instructions="You must use the tool in the same message twice",
            tools=[AnotherCustomTool])

Sync example

In [ ]:
agency = Agency([
    ceo,
], async_mode=None)  # Disable asynchronous tools mode

In [ ]:
%%time
print(agency.get_completion('Use 2 another custom tools at the same time'))

Async example

In [ ]:
agency = Agency([
    ceo,
], async_mode='tools_threading')  # Enable asynchronous tools mode

In [ ]:
%%time
print(agency.get_completion('Use 2 another custom tools at the same time'))

# Parallel Tool Calling
Sometimes, you might want your agent to use all or some of the tools sequentially. For example, when you want your agents to see the results of the previous action before proceeding with the next one. For this you have 2 options:

## `parallel_tool_calls`
You can specify weather to run tools in parallel or sequentially by setting the `parallel_tool_calls` parameter. By default, this parameter is set to `True`. See [OpenAI Docs](https://platform.openai.com/docs/api-reference/runs/createRun#runs-createrun-parallel_tool_calls)

In [ ]:
ceo = Agent(name="CEO",
            instructions="You must use the tool in the same message twice",
            tools=[AnotherCustomTool],
            parallel_tool_calls=False)

In [ ]:
agency = Agency([
    ceo,
], async_mode='tools_threading')  # Enable asynchronous tools mode

In [ ]:
%%time
print(agency.get_completion('Use call another custom tools twice at the same time'))

## `one_call_at_a_time`
Prevent multiple instances of the same **specific** tool from running at the same time.

Pre made agent example

In [ ]:
!agency-swarm import-agent --name Devid

Simple example

In [ ]:
import csv
import os
import time
from agency_swarm.tools import BaseTool
from pydantic import Field, model_validator
from typing import ClassVar

class CreateCsvSheetTool(BaseTool):
    file_path: str = Field(..., description="The path to create the CSV file.")
    headers: list = Field(..., description="The headers for the CSV file.")

    def run(self):
        time.sleep(5)  # Simulate processing time
        with open(self.file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(self.headers)
        self.shared_state.set('csv_file_path', self.file_path)
        return f"CSV file created at {self.file_path} with headers {self.headers}"

class FillRowInCsvTool(BaseTool):
    row: list = Field(..., description="The row to add to the CSV file.")
    one_call_at_a_time: bool = True

    @model_validator(mode='after')
    def check_csv_exists(self):
        file_path = self.shared_state.get('csv_file_path')
        if not file_path or not os.path.exists(file_path):
            raise ValueError("CSV file does not exist. Please create the CSV file first.")
        return self

    def run(self):
        time.sleep(5)  # Simulate processing time
        file_path = self.shared_state.get('csv_file_path')
        with open(file_path, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(self.row)
        return f"Row {self.row} added to CSV file at {file_path}"

In [ ]:
csv_sheet_agent = Agent(name="CSV Sheet Agent",
                        description="Responsible for creating and filling CSV sheets.",
                        tools=[CreateCsvSheetTool, FillRowInCsvTool])  # CSV Sheet Agent with tools

In [ ]:
agency = Agency([csv_sheet_agent], temperature=0)

In [ ]:
print(agency.get_completion('add any example row to csv file. Go'))

In [ ]:
print(agency.get_completion('Run fill row tool twice together and add more rows.'))

# Advanced `Agent` Params and Tecniques
All parameters inside the Agent class, primarily follow the same structure as OpenAI's Assistants API. However, there are a few additional parameters that you can use to customize your agent.

## Examples
You can now also provide **few-shot** examples for each agent. These examples help the agent to understand how to respond. The format for examples follows [message object format on OpenAI](https://platform.openai.com/docs/api-reference/messages/createMessage):

In [ ]:
few_shot_examples = [
    {
        "role": "user",
        "content": "Can you tell me more about the capabilities of your AI agents?",
    },
    {
        "role": "assistant",
        "content": "Absolutely! 😊 Our AI agents can handle various tasks, including data processing, customer support, and automating repetitive tasks. They integrate seamlessly with tools like Slack, email, and APIs to optimize your workflow. 🤖💼📧",
    },
    {
        "role": "user",
        "content": "What is the status of my subscription setup?",
    },
    {
        "role": "assistant",
        "content": "Your subscription setup is currently in progress. 🛠️ It should be fully activated within the next 2 business days. 🚀📈",
    },
    {
        "role": "user",
        "content": "How can I provide feedback on the AI agent deployed in my company?",
    },
    {
        "role": "assistant",
        "content": "To provide feedback, please visit our feedback page and fill out the form. 📄 We appreciate your input and strive to improve our services based on your suggestions. 📝🙏",
    }
]


In [ ]:
customer_service_agent = Agent(name="Customer Service Agent",
                               temperature=0,
                               examples=few_shot_examples)  # Adding few-shot examples to the agent


In [ ]:
agency = Agency([
    customer_service_agent,
])

In [ ]:
print(agency.get_completion("How can I customize the AI agent's responses?"))

## Response Validators
You can also provide a response validator function to validate the response before sending it to the user or another agent. This function should raise an error if the response is invalid.

In [ ]:
class CustomerServiceAgent(Agent):
  def __init__(self):
    super().__init__(name="Customer Service Agent",
                     description="Handles inquiries about products, order status, and returns.",
                     examples=few_shot_examples,
                     validation_attempts=1)

  def response_validator(self, message: str) -> str:
      if '😊' not in message or '❤️' not in message or '📱' not in message or '🛠️' not in message or '🚚' not in message or '📦' not in message or '📄' not in response or '📧' not in message:
          raise ValueError("Response does not contain required emojis!!!!!!")
      return message # you can also modify it

In [ ]:
customer_service_agent = CustomerServiceAgent()

In [ ]:
agency = Agency([
    customer_service_agent
])

In [ ]:
print(agency.get_completion("Tell me about the features of the Laptop Pro 15. Please don't output any emojis, I hate emojis"))

# Fine Tuned Models

You can use any previously fine-tuned model by specifying the `model` parameter in the agent.

In [ ]:
email_agent = Agent(
    name="Email Agent",
    description="Responds to emails.",
    model="ft:gpt-3.5-turbo:your_org:name:7rOSGOQm",  # Fine-tuned model
)

## File Search Config

You can also specify the file search configuration for the agent, as described in the [OpenAI documentation](https://platform.openai.com/docs/api-reference/assistants/createAssistant#assistants-createassistant-tools). Right now, only `max_num_results` is supported.


In [ ]:
agent = Agent(name='MyAgent',
              files_folder="./my_files",
              file_search={'max_num_results': 25} # must be between 1 and 50
              )

## Truncation Params

In [ ]:
agent = Agent(
    name="ExampleAgent",
    temperature=0.5,
    max_prompt_tokens=2048,
    max_completion_tokens=16000,
    truncation_strategy='auto',
    response_format={"type": "json_object"}
)

Also you can set in Agency class, which will act as defaults. Agent parameters will always take precedence.

In [ ]:
agency = Agency(
    agency_chart=[
        ceo,
    ],
    temperature=0.5,  # setting default temperature for the agency
    max_prompt_tokens=2500,  # setting default max_prompt_tokens for the agency
    max_completion_tokens=750,  # setting default max_completion_tokens for the agency
    truncation_strategy='auto',  # setting default truncation_strategy for the agency
    response_format={"type": "json_object"},  # setting default response_format for the agency
)

# `ToolFactory` Open API Schemas

This is the most underrated feature of my framework. Tool facotry converts any OpenAPI schema into `BaseTool`s, which allows your agents to validate all inputs **before** calling the API, significantly reducing the production errors.

### Schemas Folder

You can specify the folder where the agent will look for OpenAPI schemas to convert into tools. Additionally, you can add `api_params` and `api_headers` to the schema to pass additional parameters and headers to the API call.

In [ ]:
agent = Agent(name='MyAgent',
              schemas_folder='schemas',
              api_params={'my_schema.json': {'param1': 'value1'}},
              api_headers={'my_schema.json': {'Authorization': 'Bearer token'}}
            )

In [ ]:
agency = Agency([agent])

In [ ]:
agency.demo_gradio()

Can also be done with `ToolFactory` class

In [ ]:
from agency_swarm.tools import ToolFactory
import requests

tools = ToolFactory.from_openapi_schema(
    requests.get("https://api.example.com/openapi.json").json(),
    headers={'Authorization': 'Bearer token'},
    params={'param1': 'value1'}
)

# Get Completion Advanced params
Lastly, there are some params that can be useful for backend integrations in `get_completion` method:



*   `additional_isntructions` - useful for passing session or user info.
*   `tool_choice` - useful for forcing the agent to perform a certain action based on other confitions.
*   `recipient_agent` - sends message to a different agent in the agency.





In [ ]:
from Devid import Devid

In [ ]:
devid = Devid()

agency = Agency(
    [
        ceo,  # CEO can communicate with the user
        [ceo, devid]  # CEO can delegate tasks to the Developer
    ],
    shared_instructions='Please always address the user by name.'
)

In [ ]:
user_name = "John Doe"

task_description = "Create a function that calculates the factorial of a number."

response = agency.get_completion(
    message=task_description,
    recipient_agent=devid,  # Specifying the Developer agent
    additional_instructions=f"User name: {user_name}",  # Passing user name
    tool_choice={"type": "function", "function": {"name": "FileWriter"}}  # Specifying the tool to use
)

print(response)

# Open Source Models with Astra Assistants API

Astra Assistants persists assistant metadata in AstraDB (managed Apache Cassandra) and uses AstraDB for ANN, [here](https://www.datastax.com/pricing/astra-db) are the details on AstrDB pricing / free tier.

This now the best and the easiest way to run open source models with my framework. It takes only 5 minutes to setup.


Please make sure to give https://github.com/datastax/astra-assistants-api a star ⭐ on github.

1. Install Astra Assistants API

In [ ]:
!pip install astra_assistants

2. Get Your Astra DB token [here](https://astra.datastax.com/).


In [ ]:
import os
from getpass import getpass

os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass("Your Astra DB Application Token: ")

3. Add other model provider API keys to your env

In [ ]:
os.environ["PERPLEXITYAI_API_KEY"] = getpass("Your Perplexity AI API Key: ")

os.environ["ANTHROPIC_API_KEY"] = getpass("Your Anthropic API Key: ")

os.environ["TOGETHER_API_KEY"] = getpass("Your Together API Key: ")

os.environ["GROQ_API_KEY"] = getpass("Your Groq API Key: ")

4. Patch the OpenAI client

In [ ]:
from openai import OpenAI
from astra_assistants import patch
from agency_swarm import get_openai_client, set_openai_client

client = patch(OpenAI())

set_openai_client(client)

5. Create your Agents

In [ ]:
from agency_swarm import Agent, Agency

groq_agent = Agent(name='GroqAgent', model="groq/llama3-8b-8192")

ceo = Agent(name='CEO',
            model="claude-3-haiku-20240307",
            files_folder='./files',
            file_search={'max_num_results': 25},
            instructions="Please answer the question as best as you can. You can use the send message tool only if users tells you to ask another agent."
            )

agency = Agency([ceo, groq_agent])

6. Run Demo

In [ ]:
demo = agency.demo_gradio()

In [ ]:
demo.close()

# Next Steps:
1. Run Open Source Model with Ollama Locally at https://github.com/VRSEN/agency-swarm-lab/tree/main/AstraOpenSourceSwarm
2. See more use cases at https://github.com/VRSEN/agency-swarm-lab/tree/main
3. Checkout docs at https://vrsen.github.io/agency-swarm/
4. Join our discord [here](https://www.youtube.com/redirect?event=channel_description&redir_token=QUFFLUhqbldDNVZlVnpkVFdoWHdueG1JMFVqbklfZUhUQXxBQ3Jtc0tuM2xKYVJ5X0VBZS1qa0t3QlVYMWNVaHJHMjdZbkNQUHhkLWswWWE2STlybVl0bDBVVFptbnlpdjBFbVAya0hxek02MnZYOVB0SFUydnk5ZkZUM2lmRFhMRUZZVW1kQnoxZ1ljcDdGamktay12NGRCMA&q=https%3A%2F%2Fdiscord.gg%2F7HcABDpFPG)